## Этап - *Парсинг данных и создание датасета*

#### Импорт библиотек
Для парсинга я использовал библиотеки: BeautifulSoup, request и hyper.

Изначально авито идентифицировал мой request запрос как запрос парсера и выдавал ошибку 403 (доступ запрещен), несмотря на изменение HEADERS и попытки подключения через мобильную сеть. Поэтому использовал hyper.contrib для обхода ошибки 403, такое решение нашел в [интернете](https://qna.habr.com/q/1124028?).

Для создание датасета использовал библиотеки: Pandas, Numpy

In [ ]:
from bs4 import BeautifulSoup
import requests
from requests import get
import time
import random
from hyper.contrib import HTTP20Adapter # Обход ошибки 403

import pandas as pd
import numpy as np

#### Обзор веб-страницы
На странице представлены div блоки с объявлением продажи квартиры. На странице представлено около 50 объявлений. С данного div блока можно достать следующую информацию:
- количество комнат;
- площадь;
- этаж и этажность дома;
- цена;
- цена за м<sup>2</sup>
- ссылка на веб-страницу объявления
- адрес
<img src="./Images/screen-main.png" width="800"/>

#### Обзор веб-страницы объявления
Перейдя по ссылка на объявление, можно достать дополнительную информацию об объекте, но эта информация добавляется пользователем самостоятельно и также некоторые позиции могут быть субъективны, поэтому разные объявления имеют разную дополнительную информацию.

| first example | second example |
| :---: | :---: |
|![](./Images/screen-inner-1.png)  |  ![](./Images/screen-inner-2.png) |

#### Отбор данных
Со страницы с объявлениями можно брать все данные, так как они идентичны для каждого объявления. Со страницы объекта недвижимости я решил отобрать объективные и информативные данные, такие как санузел, балкон, год постройки.

In [ ]:
HEADERS = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15'}

rooms = []
prices = []
areas = []
floors = []
max_floors = []
links = []
balcony_or_loggia = []
bathroom = []
built_year = []

In [ ]:
def main_request(url):
    '''Запрос страницы с сайта'''
    s = requests.Session()
    s.mount('https://', HTTP20Adapter())
    response = s.get(url, headers=HEADERS)
    return response


def select_blocks(url, tag, class_name):
    '''Отбор блоков объявлений'''
    html_soup = BeautifulSoup(main_request(url).content, 'html.parser')
    house_data = html_soup.find_all(tag, class_=class_name)
    return house_data


def select_features(house_data):
    '''Добавление признака в соответствующий список'''
    count_inner = 0
    while count_inner < len(house_data): 
        house = house_data[count_inner]

        # Определение стоимости квартиры
        price = house.find('span', {'class': 'price-text-_YGDY text-text-LurtD text-size-s-BxGpL'}).text
        price = ''.join(price.split()[:-1])

        # Сбор строчек с заголовка (кол-во комнат, метраж, этаж)
        if house.find('a', {'class': 'link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH'}):
            title = house.find('a', {'class': 'link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH'}).text
        elif house.find('a', {'class': 'link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-large-jkTpZ title-root_maxHeight-X6PsH'}):
            title = house.find('a', {'class': 'link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-large-jkTpZ title-root_maxHeight-X6PsH'}).text
        else:
            print('Ошибка поиска title')
        title = [i.strip(',') for i in title.split()]

        # Добавление признаков в соответсвующие списки
        if title[1] == 'квартира':                  # Если заголовок имеет кол-во комнат
            rooms.append(title[0][0])
            area = '.'.join(title[2].split(','))
            floor = title[4].split('/')[0]
            max_floor = title[4].split('/')[1]
        else:                                       # Если заголовок "квартира-студия"
            rooms.append(1)                         # Если квартира-студия, то пусть комнат будет 1
            area = '.'.join(title[1].split(','))
            floor = title[3].split('/')[0]
            max_floor = title[3].split('/')[1]

        prices.append(price)
        areas.append(area)
        floors.append(floor)
        max_floors.append(max_floor)

        link = 'https://www.avito.ru' + house.find('a')['href']
        links.append(link)
        
        # Запрос со страницы объекта недвижимости и отбор признаков
        delay()
        house_info = select_blocks(link, page_tag, page_class_name)
        
        bal_or_log = ''
        bath = ''
        year = ''
        for i in house_info:
            if 'Балкон' in i.find('span', {'class': 'item-params-label'}).text:
                bal_or_log = i.text.split()[-1]
            if 'Санузел' in i.find('span', {'class': 'item-params-label'}).text:
                bath = i.text.split()[-1]
            if 'Год' in i.find('span', {'class': 'item-params-label'}).text:
                year = i.text.split()[-1]
        if bal_or_log:
            balcony_or_loggia.append(bal_or_log)
        else:
            balcony_or_loggia.append(np.NaN)
        if bath:
            bathroom.append(bath)
        else:
            bathroom.append(np.NaN)
        if year:
            built_year.append(year)
        else:
            built_year.append(np.NaN)
        count_inner += 1
        return None
        
def delay():
    '''Добавление задержки перед следующим запросом'''
    value = random.random()
    scaled_value = 1 + (value * (9 - 7))
    #print(scaled_value)
    time.sleep(scaled_value)
    return None


main_tag = 'div'
main_class_name = 'iva-item-content-rejJg'
page_tag = 'li'
page_class_name = 'item-params-list-item'
count = 1



while count <= 65: # кол-во страниц с объявлениями
    startTime = time.time()
    url = 'https://www.avito.ru/perm/kvartiry/prodam/vtorichka-ASgBAgICAkSSA8YQ5geMUg?context=H4sIAAAAAAAA_0q0MrSqLraysFJKK8rPDUhMT1WyLrYyNLNSKk5NLErOcMsvyg3PTElPLVGyrgUEAAD__xf8iH4tAAAA&p=' + str(count)
    select_features(select_blocks(url, main_tag, main_class_name))
    
    endTime = time.time() 
    totalTime = endTime - startTime 
    print(f'Время выполнения блока {count} = {totalTime:.2f}')
    
    count += 1

In [ ]:
df = pd.DataFrame({'Price': prices, 
                   'Rooms': rooms, 
                   'Area': areas, 
                   'Floor': floors, 
                   'Max_house_floor': max_floors, 
                   'balcony\loggia': balcony_or_loggia,
                   'Bathroom': bathroom,
                   'Year built': built_year,
                   'Link': links})

In [ ]:
df.to_csv('database.csv', encoding='utf-8')